# モデルの評価指標を集約して、DataFrameに格納する

In [ ]:
import sys
import os
import boto3
import pandas as pd

In [ ]:
REGION = "ap-northeast-1"

In [ ]:
forecast = boto3.client('forecast', region_name= REGION)

https://boto3.amazonaws.com/v1/documentation/api/1.12.6/reference/services/forecast.html#ForecastService.Client.list_predictors

In [ ]:
list_predictors = forecast.list_predictors()

In [ ]:
len(list_predictors['Predictors'])

describe_predictor()

https://boto3.amazonaws.com/v1/documentation/api/1.12.6/reference/services/forecast.html#ForecastService.Client.describe_predictor

get_accuracy_metrics()

https://boto3.amazonaws.com/v1/documentation/api/1.12.6/reference/services/forecast.html#ForecastService.Client.get_accuracy_metrics


## 1列ずつリストで作成して、DataFrame化する

参考：https://qiita.com/studio_haneya/items/35951c56decd212ba41e

50倍速い！

In [ ]:
# 情報格納リストの初期化
l_PredictorArn=[]
l_PredictorName=[]
l_AverageWeightedQuantileLoss=[]
l_RMSE=[]
l_q90_LossValue=[]
l_q50_LossValue=[]
l_q10_LossValue=[]
l_WAPE=[]
l_MASE=[]
l_MAPE=[] 
l_context_length=[]
l_epochs=[]
l_learning_rate=[]
l_learning_rate_decay=[]
l_likelihood=[]
l_max_learning_rate_decays=[]
l_num_averaged_models=[]
l_num_cells=[]
l_num_layers=[]
l_prediction_length=[]


for i in range(len(list_predictors['Predictors'])):
    print(list_predictors['Predictors'][i]['PredictorArn'])
    try:
        # describe_predictor()
        desc_pred = forecast.describe_predictor(PredictorArn=list_predictors['Predictors'][i]['PredictorArn'])
        
        if desc_pred['AutoMLAlgorithmArns'][0]!='arn:aws:forecast:::algorithm/Deep_AR_Plus':
            print("not DeepAR+")
            desc_pred = prev_desc_pred
            continue
        if desc_pred['PerformAutoML']==True:
            print("AutoML was used")
            desc_pred = prev_desc_pred
            continue
            
        prev_desc_pred = desc_pred ### 取りたい対象のdesc_predを補完(上if文の戻しで利用)

        # describe_predictor()
        acc_pred = forecast.get_accuracy_metrics(PredictorArn=list_predictors['Predictors'][i]['PredictorArn'])
        
    except:
        continue
        
    # 数値を取得して、リストに格納
    l_PredictorArn+=[list_predictors['Predictors'][i]['PredictorArn']]
    l_PredictorName+=[desc_pred['PredictorName']]
    l_AverageWeightedQuantileLoss+=[acc_pred['PredictorEvaluationResults'][0]['TestWindows'][0]['Metrics']['AverageWeightedQuantileLoss']]
    l_RMSE+=[acc_pred['PredictorEvaluationResults'][0]['TestWindows'][0]['Metrics']['RMSE']]
    l_q90_LossValue+=[acc_pred['PredictorEvaluationResults'][0]['TestWindows'][0]['Metrics']['WeightedQuantileLosses'][0]['LossValue']]
    l_q50_LossValue+=[acc_pred['PredictorEvaluationResults'][0]['TestWindows'][0]['Metrics']['WeightedQuantileLosses'][1]['LossValue']]
    l_q10_LossValue+=[acc_pred['PredictorEvaluationResults'][0]['TestWindows'][0]['Metrics']['WeightedQuantileLosses'][2]['LossValue']]
    l_WAPE+=[acc_pred['PredictorEvaluationResults'][0]['TestWindows'][0]['Metrics']['ErrorMetrics'][0]['WAPE']]
    l_MASE+=[acc_pred['PredictorEvaluationResults'][0]['TestWindows'][0]['Metrics']['ErrorMetrics'][0]['MASE']]
    l_MAPE+=[acc_pred['PredictorEvaluationResults'][0]['TestWindows'][0]['Metrics']['ErrorMetrics'][0]['MAPE']]
    l_context_length+=[desc_pred['TrainingParameters']['context_length']]
    l_epochs+=[desc_pred['TrainingParameters']['epochs']]
    l_learning_rate+=[desc_pred['TrainingParameters']['learning_rate']]
    l_learning_rate_decay+=[desc_pred['TrainingParameters']['learning_rate_decay']]
    l_likelihood+=[desc_pred['TrainingParameters']['likelihood']]
    l_max_learning_rate_decays+=[desc_pred['TrainingParameters']['max_learning_rate_decays']]
    l_num_averaged_models+=[desc_pred['TrainingParameters']['num_averaged_models']]
    l_num_cells+=[desc_pred['TrainingParameters']['num_cells']]
    l_num_layers+=[desc_pred['TrainingParameters']['num_layers']]
    l_prediction_length+=[desc_pred['TrainingParameters']['prediction_length']]


In [ ]:
acc_pred['PredictorEvaluationResults'][0]['TestWindows'][0]['Metrics']

In [ ]:
acc_pred['PredictorEvaluationResults'][0]['TestWindows'][0]['Metrics']['RMSE']
acc_pred['PredictorEvaluationResults'][0]['TestWindows'][0]['Metrics']['WeightedQuantileLosses'][0]['LossValue']
acc_pred['PredictorEvaluationResults'][0]['TestWindows'][0]['Metrics']['WeightedQuantileLosses'][1]['LossValue']
acc_pred['PredictorEvaluationResults'][0]['TestWindows'][0]['Metrics']['WeightedQuantileLosses'][2]['LossValue']
acc_pred['PredictorEvaluationResults'][0]['TestWindows'][0]['Metrics']['ErrorMetrics'][0]['WAPE']
acc_pred['PredictorEvaluationResults'][0]['TestWindows'][0]['Metrics']['ErrorMetrics'][0]['MASE']
acc_pred['PredictorEvaluationResults'][0]['TestWindows'][0]['Metrics']['ErrorMetrics'][0]['MAPE']

### リスト化していた情報を列にして、DataFrameを作成

In [ ]:
df = pd.DataFrame(data={'PredictorArn': l_PredictorArn,
                        'PredictorName': l_PredictorName,
                        'AverageWeightedQuantileLoss': l_AverageWeightedQuantileLoss,
                        'RMSE': l_RMSE,
                        'q90_LossValue': l_q90_LossValue,
                        'q50_LossValue': l_q50_LossValue,
                        'q10_LossValue': l_q10_LossValue,
                        'WAPE': l_WAPE,
                        'MASE': l_MASE,
                        'MAPE': l_MAPE, 
                        'context_length': l_context_length,
                        'epochs': l_epochs,
                        'learning_rate': l_learning_rate,
                        'learning_rate_decay': l_learning_rate_decay,
                        'likelihood': l_likelihood,
                        'max_learning_rate_decays': l_max_learning_rate_decays,
                        'num_averaged_models': l_num_averaged_models,
                        'num_cells': l_num_cells,
                        'num_layers': l_num_layers,
                        'prediction_length': l_prediction_length
                       }
    )

In [ ]:
print(df.shape)
df

In [ ]:
# 確認
desc_pred

In [ ]:
# 確認
acc_pred